In [1]:
# Imports
%matplotlib inline
import numpy as np
from tqdm import tqdm
from tensorflow.keras import layers, models
import tensorflow as tf
import networkx as nx
from tabulate import tabulate
import time

## Graph Environment

In [2]:
class GraphEnvironment:
    def __init__(self, graph):
        self.graph = graph  # The graph represented as an adjacency list
        self.num_nodes = len(graph)
        self.global_state = self.initialize_global_state()  # State of every node -> 0 (will be decided), 1 (included), 2 (not included)
        self.state = self.initialize_state()    # An adjacency list representing State of every agent as state of the node along with state of all its neighbours

    ## Function to initialize self.state as all zeros
    def initialize_state(self):
        z = [[0]*(len(row)+1) for row in self.graph]
        return z

    ## Function to initialize self.global_state as all zeros
    def initialize_global_state(self):
        return np.zeros(self.num_nodes)
    
    ## Helper function for the remove_adjacent function
    def bfs(self, ind, s):
        vis = np.zeros(self.num_nodes)
        q = []
        vis[ind]=1
        f = False
        q.append(ind)
        ## Additional reward which is included as negative reward to punish the agents for choosing adjacent nodes to be included in the set 
        rew = 1
        while len(q):
            i = q.pop(0)
            for j in self.graph[i]:
                if(vis[j]==0 and s[j]==1):
                    f = True
                    rew += 1
                    s[j]=0
                    q.append(j)
                    
        if f:   s[ind] = 0
        return rew
    
    ## Function to set state of every pair of adjacent nodes whose state has been changed to included
    def remove_adjacent(self, s):
        ## Helper function to help remove all connected nodes which have been set to be included
        rew = 0
        for i in range(self.num_nodes):
            if s[i] == 1 :
                rew += self.bfs(i,s)
        return s, rew

    ## Function to include all nodes in set which are surrounded by all not included nodes as it is always optimal to include this node in set
    def include_possible(self, s):
        for i in range(self.num_nodes) :
            f = True
            if s[i] == 0 :
                for j in self.graph[i] :
                    if s[j] in [0, 1] :
                        f = False
                        break
                if f : s[i] = 1
        return s

    ## Function to set state of all nodes adjacent to those nodes whose state has been set to be included as not included as taking the nodes which are included they can never be included in the independent set
    def exclude_adjacent(self, s):
        for i in range(self.num_nodes):
            if s[i]==1:
                for j in self.graph[i]:
                    if s[j]==0:    s[j]=2
        return s

    ## Function calculates reward of a step in the environment as the change in cardinality of the current independent set, hence including as many nodes as possible gives more reward and trains the agents to do the same
    def calculate_reward(self, s1, s2):
        sum1=0
        sum2=0
        for i in range(self.num_nodes):
            if s2[i] == 1:
                sum2 += 1
            if s1[i] == 1:
                sum1 += 1
        return sum2 - sum1
    
    ## Returns the termination flag after every step
    def check_termination(self, s):
        for i in range(self.num_nodes):
            if s[i] == 0:
                return False
        return True
    
    ## Function to convert the global state to local state representing the individual states of every agent
    def convert_global_local(self, global_s):
        local = []
        for v in range(self.num_nodes):
            temp = [global_s[v]]
            for neighbour in self.graph[v]:
                temp.append(global_s[neighbour])
            local.append(temp)
        return local

    ## Function to take the step based on action provide in the environment 
    def step(self, actions):
        global_state0 = actions
        global_state1, neg_reward = self.remove_adjacent(global_state0)
        global_state2 = self.exclude_adjacent(global_state1)
        global_state3 = self.include_possible(global_state2)
        ## We have given an additional -1 reward for every step to encourage the agent to find the MIS in as few steps as possible
        reward = self.calculate_reward(self.global_state, global_state3) - neg_reward - 1
        self.global_state = global_state3
        self.state = self.convert_global_local(self.global_state)
        done = self.check_termination(self.global_state)
        return self.state, reward, done

    # Reset the environment to the initial state
    def reset(self):
        self.global_state = self.initialize_global_state()
        self.state = self.initialize_state()
        return self.state

## Testing the graph environment 

In [3]:
graph = np.array([
    [1, 2, 5],
    [0, 2, 3],
    [0, 1, 3, 4],
    [1, 2, 4, 5],
    [2, 3, 5],
    [0, 3, 4]
],dtype=object)

env = GraphEnvironment(graph)


In [4]:
env.reset()
s, rew, done = env.step(np.array([2,2,0,2,2,0]))
print("Reward -> ",rew)
print("Terminated -> ",done)
print(env.state)

Reward ->  1
Terminated ->  True
[[2, 2, 1, 1], [2, 2, 1, 2], [1, 2, 2, 2, 2], [2, 2, 1, 2, 1], [2, 1, 2, 1], [1, 2, 2, 2]]


## PPO Agent class 

In [5]:
## Every PPO Agent contains 2 networks, one is actor which gives the policy according to which action is to be taken and second is critic which predicts the value of being in a state
class PPOAgent:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim

        # Hyper-parameters
        self.gamma = 0.99  # Discount factor
        self.epsilon = 0.2  # PPO clip ratio
        self.lr_actor = 0.001  # Actor learning rate
        self.lr_critic = 0.001  # Critic learning rate

        # Build actor and critic networks
        self.actor = self.build_actor()
        self.critic = self.build_critic()

        # Optimizers
        self.actor_optimizer = tf.keras.optimizers.Adam(learning_rate=self.lr_actor)
        self.critic_optimizer = tf.keras.optimizers.Adam(learning_rate=self.lr_critic)

    ## Neural Network model builder for the actor model containing a single hidden layer with 64 nodes
    def build_actor(self):
        model = models.Sequential([
            layers.InputLayer(input_shape=(self.state_dim,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(self.action_dim, activation='softmax')
        ])
        return model
    
    ## Neural Network model builder for the critic model containing a single hidden layer with 64 nodes
    def build_critic(self):
        model = models.Sequential([
            layers.InputLayer(input_shape=(self.state_dim,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')
        ])
        return model
    
    ## Function to sample actions for the agent based on policy from the actor
    def select_action(self, state):
        action_probs = self.actor(tf.convert_to_tensor(np.expand_dims(state, axis=0)))[0]
        sample = tf.random.uniform(shape=(1,), minval=0, maxval=1)[0] 
        cum_prob = 0
        for i, p in enumerate(action_probs):
            if sample <= cum_prob + p:
                return i
            cum_prob += p


In [6]:
## Function to return discounted rewards for a episode
def discounted_rewards(rewards, gamma=0.99):
        r = np.array(rewards)
        discounted_r = np.zeros_like(r)
        running_add = 0
        for t in reversed(range(len(r))):
            running_add = running_add * gamma + r[t]
            discounted_r[t] = running_add
        return discounted_r


## Training the Multi Agent Model

In [7]:
def train_multi_agent(env, agents, num_episodes, epochs = 10, clip_epsilon = 0.2):
    for episode in range(num_episodes):
        
        ## Getting an episode/trajectory
        state = env.reset()
        all_states, all_actions, all_rewards = [], [], []
        done = False
        steps = 0
        while (not done) and steps<50:
            steps += 1
            actions = []
            all_states.append(state)
            for agent_id, agent in enumerate(agents):
                if state[agent_id][0] !=0:
                    action = state[agent_id][0]
                else:
                    action = agent.select_action(state[agent_id])
                actions.append(action)

            next_states, reward, done = env.step(actions)
            state = next_states
            all_rewards.append(reward)
            all_actions.append(actions)

        all_rewards = tf.convert_to_tensor(all_rewards, dtype=tf.float32)
        
        ## Optimizing agent parameters
        for agent_id, agent in enumerate(agents):
            with tf.GradientTape() as tape_critic:
                ## Probabilities according to policy based on which trajectory is chosen
                probs_old = tf.stack([tf.gather(agent.actor(tf.convert_to_tensor(np.expand_dims(st[agent_id], axis=0)))[0], at[agent_id]) for st, at in zip(all_states, all_actions)])
                
                ## Used the agent critic model to predict the value function for the states in the trajectory
                values = tf.convert_to_tensor([agent.critic(tf.convert_to_tensor(np.expand_dims(st[agent_id], axis=0)))[0, 0] for st in all_states])
                disc_rewards = tf.convert_to_tensor(discounted_rewards(all_rewards))
                advantages = disc_rewards - values  ## Getting Advantages array
                combined_loss = 0   ## Overall loss used to optimize critic parameters
                for _ in range(epochs):
                    with tf.GradientTape() as tape_actor:
                        ## Updated Policy
                        probs_new = tf.stack([tf.gather(agent.actor(tf.convert_to_tensor(np.expand_dims(st[agent_id], axis=0)))[0], at[agent_id]) for st, at in zip(all_states, all_actions)])
                        ratio = probs_new / probs_old   ## Policy ratio
                        clipped_ratio = tf.clip_by_value(ratio, 1 - clip_epsilon, 1 + clip_epsilon) ## Clipped policy ratio
                        policy_loss = tf.reduce_mean(tf.minimum(ratio * advantages, clipped_ratio * advantages))
                        value_loss = tf.reduce_mean(tf.keras.losses.MSE(disc_rewards, values))
                        total_loss = policy_loss + value_loss
                        combined_loss += total_loss
                    ## Calculating gradients and descending gradients
                    grads_actor = tape_actor.gradient(total_loss, agent.actor.trainable_weights)
                    agent.actor_optimizer.apply_gradients(zip(grads_actor, agent.actor.trainable_weights))
            grads_critic = tape_critic.gradient(combined_loss, agent.critic.trainable_weights)
            agent.critic_optimizer.apply_gradients(zip(grads_critic, agent.critic.trainable_weights))

In [8]:
# Code block to test training on custom graph
# graph = np.array([
#     [1, 2, 5],
#     [0, 2, 3],
#     [0, 1, 3, 4],
#     [1, 2, 4, 5],
#     [2, 3, 5],
#     [0, 3, 4]
# ],dtype=object)
# 
# env = GraphEnvironment(graph)
# 
# agents = [PPOAgent(len(v)+1, 3) for v in env.graph]
# num_episodes = 100
# 
# train_multi_agent(env, agents, num_episodes, epochs=5)

In [9]:
## Function to use the agents trained during training to sample multiple solutions to return the best among them as the prediction of the reinforcement learning algorithm
def find_MIS(env, agents, num_solutions=20):
    mis = 0
    for episode in range(num_solutions):
        state = env.reset()
        done = False
        steps = 0
        while (not done) and steps<50:
            steps += 1
            actions = []
            for agent_id, agent in enumerate(agents):
                if state[agent_id][0] !=0:
                    action = state[agent_id][0]
                else:
                    action = agent.select_action(state[agent_id])
                actions.append(action)
    
            state, reward, done = env.step(actions)
        mis = max(mis,env.global_state.count(1))
    return mis

## Testing the algorithm

In [10]:
## Function to combine all the steps in Multi agent RL algorithm
def MARL_MIS_algorithm(G, num_episodes=100, epochs=10, num_solutions=20):
    env = GraphEnvironment(G)
    agents = [PPOAgent(len(v)+1, 3) for v in env.graph]
    train_multi_agent(env, agents, num_episodes, epochs=epochs)
    return find_MIS(env, agents, num_solutions)

In [11]:
## Function to generate random graphs with the help of NetworkX library
def generate_random_graph(num_nodes, edge_prob, seed):
    g = nx.gnp_random_graph(num_nodes, edge_prob, seed=  seed)
    adj_list = [list(g.neighbors(node)) for node in g.nodes()]
    return adj_list

### Function to return greedy MIS 

In [12]:
def greedy_solution(G):
    independent_set = []
    remaining_vertices = set(range(len(G)))

    while remaining_vertices:
        v = remaining_vertices.pop()
        independent_set.append(v)
        neighbors_to_remove = set(G[v])
        remaining_vertices.difference_update(neighbors_to_remove)
        remaining_vertices.discard(v) 

    return len(independent_set)

### Function to return exact MIS using brute force

In [13]:
def is_independent_set(vertices, adj_list):
    for i in vertices:
        for j in vertices:
            if i != j and j in adj_list[i]:
                return False
    return True

def find_maximum_independent_set(current_set, index, adj_list, best_set):
    if index == len(adj_list):
        if is_independent_set(current_set, adj_list) and len(current_set) > len(best_set[0]):
            best_set[0] = current_set.copy()
        return
    
    current_set.add(index)
    find_maximum_independent_set(current_set, index + 1, adj_list, best_set)
    current_set.remove(index)
    find_maximum_independent_set(current_set, index + 1, adj_list, best_set)

def exact_solution(adj_list):
    best_set = [set()]
    find_maximum_independent_set(set(), 0, adj_list, best_set)
    return len(best_set[0])

In [14]:
def testing(num_graphs=10, num_nodes=10, edge_prob=0.5, num_episodes=100, epochs=15, num_solutions=20):
    data = []
    for i in tqdm(range(num_graphs)):
        G = generate_random_graph(num_nodes,edge_prob,i)
        greedy_mis = greedy_solution(G)
        MARL_mis = MARL_MIS_algorithm(G, num_episodes=num_episodes, epochs=epochs, num_solutions= num_solutions)
        exact_mis = exact_solution(G)
        data.append((greedy_mis,MARL_mis,exact_mis))
        
    headers = ["Greedy MIS", "MARL MIS", "Exact MIS"]
    print(tabulate(data, headers=headers, tablefmt='grid'))

In [17]:
testing(num_graphs=5, num_nodes=10, edge_prob=0.5, num_episodes=50, epochs=10, num_solutions=5)

100%|██████████| 5/5 [24:10<00:00, 290.08s/it]

+--------------+------------+-------------+
|   Greedy MIS |   MARL MIS |   Exact MIS |
+==============+============+=============+
|            4 |          3 |           4 |
+--------------+------------+-------------+
|            3 |          3 |           3 |
+--------------+------------+-------------+
|            3 |          4 |           4 |
+--------------+------------+-------------+
|            4 |          4 |           4 |
+--------------+------------+-------------+
|            3 |          4 |           5 |
+--------------+------------+-------------+
